In [62]:
import json
import os
import time
import pyautogui
from google import genai
from google.genai import types
from PIL import Image, ImageGrab
from pywinauto.findwindows import find_element

import os
import json
import cv2
import ultralytics
from google import genai
from google.genai import types
from PIL import Image
from google.generativeai.types import HarmCategory, HarmBlockThreshold



# --- API Key Handling for Local Environment ---
# Get the API key from an environment variable
API_KEY = os.getenv('GOOGLE_API_KEY')

if API_KEY:
    # Initialize the Gemini client with your API key
    client = genai.Client(api_key=API_KEY)
    print("✅ Gemini client initialized successfully.")
else:
    print("🛑 API Key not found. Please set the GOOGLE_API_KEY environment variable and restart the kernel.")

# A simpler, but less secure alternative (uncomment below if you prefer)
# WARNING: Do not share your notebook if you paste your key directly.
# API_KEY = "YOUR_API_KEY_HERE"
# client = genai.Client(api_key=API_KEY)





✅ Gemini client initialized successfully.


In [ ]:
def inference(image: Image.Image, prompt: str, temp: float = 0.2) -> str:
    """
    Performs inference using the client.models.generate_content method,
    which is the correct structure for your environment.
    """
    if not client:
        raise ValueError("Gemini client is not initialized.")

    try:
        
        generation_config = genai.types.GenerationConfig(
            temperature=temp,
            # We must rely on prompt engineering for JSON as this old method might not
            # support response_mime_type in the config.
        )

        # THE CORRECT API CALL, based on our debug results and your working example.
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[prompt, image],
            config=types.GenerateContentConfig(
                temperature=temp,
            ),
        )
        
        # Clean the response just in case it's wrapped in markdown
        cleaned_text = response.text.strip().replace("```json", "").replace("```", "").strip()
        return cleaned_text
    
    except Exception as e:
        print(f"An error occurred during Gemini API call: {e}")
        return ""

In [16]:
# Test case
def test_inference():
    # 1. Load a test image (replace with your image path)
    test_image = Image.open("masaustu.png")  # or create a blank image: Image.new('RGB', (100, 100))
    
    # 2. Define a test prompt
    test_prompt = "Describe this image in detail."
    
    # 3. Call the function
    result = inference(image=test_image, prompt=test_prompt, temp=0.2)
    
    # 4. Print the result
    print("Gemini Response:")
    print(result)



In [17]:

test_inference()

Gemini Response:
This image displays a computer desktop, likely running Windows 11, characterized by a clean, dark aesthetic.

**Overall Appearance:**
The dominant feature is the desktop background, which is a solid, uniform dark gray or charcoal color, providing a minimalist backdrop. There are no patterns, images, or gradients on the wallpaper.

**Desktop Icons (Left Side):**
A series of application shortcuts, files, and folders are neatly arranged in columns along the left side of the screen. From top to bottom, left to right:

*   **Row 1:**
    *   A standard "Recycle Bin" icon (blue bin with recycling arrows).
    *   A red PDF icon labeled "SD-WEB 2C Kurulum Pr..." (likely a setup file).
    *   A Microsoft Edge browser icon (blue-green swirl) labeled "logo-icon...".
    *   A purple icon with wavy lines, labeled "LM Studio".
*   **Row 2:**
    *   A yellow icon with a white checkmark, labeled "Norton 360".
    *   A generic white document icon labeled "Application...".
    *   

In [52]:
# Re-run this cell
def take_screenshot() -> Image.Image:
    """Takes a screenshot and returns it as a PIL Image object."""
    screenshot = ImageGrab.grab()
    print("📸 Screenshot captured.")
    return screenshot

def get_center_point(box_coords):
    """Calculates the center point of a bounding box."""
    y1, x1, y2, x2 = box_coords
    center_x = x1 + (x2 - x1) // 2
    center_y = y1 + (y2 - y1) // 2
    return  center_y, center_x

def perform_action_on_location(location_data_json: str, action: str, **kwargs):
    """
    Parses location data from the AI and performs a specified action on the GUI.
    """
    try:
        data = json.loads(location_data_json)
        box = data.get("box_2d")
        description = data.get("description", "Unknown Element")
        if not box or len(box) != 4:
            print(f"Error: Invalid 'box_2d' data in JSON: {location_data_json}")
            return
        center_x, center_y = get_center_point(box)
        print(f"🤖 Action '{action}' on '{description}' at ({center_x}, {center_y})")
        action = action.lower()
        if action == 'hover': pyautogui.moveTo(center_x, center_y, duration=0.25)
        elif action == 'click': pyautogui.click(center_x, center_y)
        elif action == 'double_click': pyautogui.doubleClick(center_x, center_y)
        elif action == 'right_click': pyautogui.rightClick(center_x, center_y)
        elif action == 'type':
            text = kwargs.get('text_to_type'); pyautogui.click(center_x, center_y); pyautogui.typewrite(text, interval=0.05)
        elif action in ['reliable_click', 'reliable_right_click', 'reliable_double_click']:
            try:
                element = find_element(coords=(center_x, center_y), backend='uia')
                if action == 'reliable_click': element.click_input()
                elif action == 'reliable_right_click': element.right_click_input()
                elif action == 'reliable_double_click': element.double_click_input()
            except Exception:
                if action == 'reliable_click': pyautogui.click(center_x, center_y)
                elif action == 'reliable_right_click': pyautogui.rightClick(center_x, center_y)
                elif action == 'reliable_double_click': pyautogui.doubleClick(center_x, center_y)
        else: print(f"Error: Unknown action '{action}'")
    except json.JSONDecodeError: print(f"Error: Invalid JSON from AI: {location_data_json}")
    except Exception as e: print(f"An unexpected error occurred: {e}")

In [54]:
def live_action_test():
    """Test with REAL mouse movements"""
    print("Testing with real mouse movements in 3 seconds...")
    print("Move mouse to corner to abort!")
    for i in range(3, 0, -1):
        print(f"{i}...", end="\r")
        time.sleep(1)
    
    # This will actually move and click
    perform_action_on_location(
        '{"box_2d": [12,1000,20,2000], "description": "test"}',
        "click"
    )
    print("Did the mouse click at (150, 150)?")

live_action_test()

Testing with real mouse movements in 3 seconds...
Move mouse to corner to abort!
🤖 Action 'click' on 'test' at (16, 1500)
Did the mouse click at (150, 150)?


In [21]:
import pyautogui

# Test basic mouse movement (watch your cursor)
pyautogui.moveTo(1000, 100, duration=1)  # Should visibly move
pyautogui.click()  # Should click at (100, 100)

In [22]:
def test_screenshot():
    try:
        img = take_screenshot()
        print(f"Screenshot captured. Size: {img.size} (should match your screen resolution)")
        # img.show()  # Uncomment to actually see the screenshot
    except Exception as e:
        print(f"Screenshot failed: {e}")

test_screenshot()

📸 Screenshot captured.
Screenshot captured. Size: (1920, 1080) (should match your screen resolution)


In [16]:
def test_center_point():
    test_cases = [
        ((0, 0, 100, 100), (50, 50)),    # Simple square
        ((10, 20, 30, 60), (20, 40)),    # Rectangle
        ((5, 5, 15, 25), (10, 15))       # Small rectangle
    ]
    
    for coords, expected in test_cases:
        result = get_center_point(coords)
        assert result == expected, f"Failed: {coords} → {result} (expected {expected})"
        print(f"✓ {coords} → {result}")

test_center_point()

✓ (0, 0, 100, 100) → (50, 50)
✓ (10, 20, 30, 60) → (20, 40)
✓ (5, 5, 15, 25) → (10, 15)


In [17]:
def test_actions():
    # Mock JSON data (what we expect from the AI)
    test_data = {
        "valid_click": ('{"box_2d": [100, 100, 200, 200], "description": "Test Button"}', "click"),
        "invalid_json": ("not a json", "hover"),
        "missing_box": ('{"description": "No coordinates"}', "click"),
        "typing_test": ('{"box_2d": [50, 50, 150, 150]}', "type", {"text_to_type": "Hello"})
    }
    
    print("\nTesting actions (will not move real mouse):")
    for name, (json_str, action, *args) in test_data.items():
        kwargs = args[0] if args else {}
        print(f"\nCase: {name}")
        print(f"Input: {json_str}")
        perform_action_on_location(json_str, action, **kwargs)

test_actions()


Testing actions (will not move real mouse):

Case: valid_click
Input: {"box_2d": [100, 100, 200, 200], "description": "Test Button"}
🤖 Action 'click' on 'Test Button' at (150, 150)

Case: invalid_json
Input: not a json
Error: Invalid JSON from AI: not a json

Case: missing_box
Input: {"description": "No coordinates"}
Error: Invalid 'box_2d' data in JSON: {"description": "No coordinates"}

Case: typing_test
Input: {"box_2d": [50, 50, 150, 150]}
🤖 Action 'type' on 'Unknown Element' at (100, 100)


In [55]:
# Re-run this cell too
def real_ai_vision_model(screen_image: Image.Image, object_to_find: str) -> str | None:
    """
    Sends the screenshot to the Gemini model to find an object and returns its location.
    """
    print(f"🧠 AI Model: Analyzing screen to find '{object_to_find}'...")
    prompt = f"""
    You are an expert AI assistant for GUI automation. Analyze the provided screenshot and locate the UI element described as: "{object_to_find}".
    Respond with ONLY a JSON object with two keys: "box_2d" and "description".
    Do not include any other text or markdown formatting. If you cannot find it, return JSON with an empty list for "box_2d".
    """
    ai_response_text = inference(screen_image, prompt) # This now calls the new function
    if not ai_response_text:
        print("AI Model: Received no response."); return None
    print(f"🔍 AI Raw Response:\n{ai_response_text}")
    try:
        response_json = json.loads(ai_response_text)
        if "box_2d" in response_json and response_json["box_2d"]:
            return ai_response_text
        else:
            print(f"AI Model: Could not find '{object_to_find}'."); return None
    except json.JSONDecodeError:
        print(f"AI Model: Failed to decode JSON from response."); return None

In [24]:
# # --- MAIN EXECUTION EXAMPLE ---
# print("Starting automation in 5 seconds. Please switch to your target window (e.g., the Desktop).")
# for i in range(5, 0, -1):
#     print(f"{i}..."); time.sleep(1)
# print("Go!")

# # --- Demo 1: Double-clicking the Recycle Bin ---
# print("\n--- Demo 1: Double-clicking the Recycle Bin ---")
# screen_image = take_screenshot()
# recycle_bin_location_json = real_ai_vision_model(screen_image, "The Recycle Bin icon")
# if recycle_bin_location_json:
#     perform_action_on_location(recycle_bin_location_json, "reliable_double_click")
# else:
#     print("Could not perform action because the Recycle Bin was not found.")

# time.sleep(5) 
# print("\nClose any open windows before the next step.")
# time.sleep(3)

# --- Demo 2: Right-clicking the Start Menu ---
print("\n--- Demo 2: Right-clicking the Start Menu ---")
screen_image_2 = take_screenshot()
start_menu_location_json = real_ai_vision_model(screen_image_2, "start menu button")
if start_menu_location_json:
    perform_action_on_location(start_menu_location_json, "right_click")
else:
    print("Could not perform action because the Start Menu was not found.")


--- Demo 2: Right-clicking the Start Menu ---
📸 Screenshot captured.
🧠 AI Model: Analyzing screen to find 'start menu button'...
🔍 AI Raw Response:
{"box_2d": [225, 946, 260, 970], "description": "start menu button"}
🤖 Action 'right_click' on 'start menu button' at (958, 242)


In [67]:
from PIL import Image, ImageDraw

# --- Demo 2: Right-clicking the Start Menu ---
print("\n--- Demo 2: Right-clicking the Start Menu ---")

# 1. Take screenshot
screen_image_2 = take_screenshot()

# 2. Find Start Menu button
start_menu_location_json = real_ai_vision_model(screen_image_2, "visual studio code icon")  # Changed to "Google Chrome icon" for testing

if start_menu_location_json:
    try:
        # 3. Visual confirmation - draw box on screenshot
        data = json.loads(start_menu_location_json)
        box = data["box_2d"]
        description = data["description"]
        
        # Create a copy of the screenshot to draw on
        marked_image = screen_image_2.copy()
        draw = ImageDraw.Draw(marked_image)
        
        # Draw red rectangle around detected area
        draw.rectangle(box, outline="red", width=3)
        
        # Add text label
        draw.text((box[0], box[1] - 20), description, fill="red")
        
        # Show the marked image
        print("\nShowing detected Start Menu button (red box)...")
        marked_image.show()
        
        # 4. Countdown before performing action
        print("\nRight-clicking Start Menu in 5 seconds...")
        for i in range(5, 0, -1):
            print(f"{i}...", end=" ", flush=True)
            time.sleep(1)
            
        # 5. Perform the right-click
        perform_action_on_location(start_menu_location_json, "right_click")
        print("\nRight-click performed!")
        
    except Exception as e:
        print(f"\nError during visualization: {str(e)}")
else:
    print("Could not perform action because the Start Menu was not found.")


--- Demo 2: Right-clicking the Start Menu ---
📸 Screenshot captured.
🧠 AI Model: Analyzing screen to find 'visual studio code icon'...
🔍 AI Raw Response:
{"box_2d": [1, 773, 85, 856], "description": "Visual Studio Code icon"}

Showing detected Start Menu button (red box)...

Right-clicking Start Menu in 5 seconds...
5... 4... 3... 2... 1... 🤖 Action 'right_click' on 'Visual Studio Code icon' at (43, 814)

Right-click performed!


In [ ]:
import pyautogui
pyautogui.FAILSAFE = False
# Test basic mouse movement (watch your cursor)
pyautogui.moveTo(90, 480, duration=1)  # Should visibly move
pyautogui.click()  # Should click at (100, 100)

In [20]:
# --- DEBUGGING CELL ---

# We assume 'client' was created in the previous cell like this:
# client = genai.GenerativeModel('gemini-1.5-flash-latest')

print("--- Starting Debug Inspection ---")
try:
    # 1. What is the actual type of our 'client' object?
    print(f"The type of our 'client' object is: {type(client)}")
    print("\n-----------------------------------\n")
    
    # 2. What methods and attributes are available on this object?
    print("Methods and attributes available on the 'client' object:")
    # The dir() function is a built-in Python tool to list all properties of an object.
    # We will print them in an easy-to-read list.
    available_attributes = dir(client)
    for attr in available_attributes:
        # We'll ignore the "dunder" (double underscore) methods for clarity
        if not attr.startswith('__'):
            print(f"- {attr}")
            
    print("\n--- Debug Analysis ---")
    if 'generate_content' in available_attributes:
        print("✅ 'generate_content' IS in the list. The error is very strange.")
    else:
        print("❌ 'generate_content' IS NOT in the list. This confirms the error.")
        print("   This means the 'client' object from 'genai.GenerativeModel()' in your environment")
        print("   is NOT the modern object we expect. It's a different, older 'Client' type.")
        print("   The fix is to stop using this object-oriented approach and use a simpler function call.")
        
except NameError:
    print("Error: The 'client' object has not been created yet. Please run the initialization cell first.")
except Exception as e:
    print(f"An unexpected error occurred during debugging: {e}")

--- Starting Debug Inspection ---
The type of our 'client' object is: <class 'google.genai.client.Client'>

-----------------------------------

Methods and attributes available on the 'client' object:
- _aio
- _api_client
- _batches
- _caches
- _debug_config
- _files
- _get_api_client
- _models
- _operations
- _tokens
- _tunings
- aio
- auth_tokens
- batches
- caches
- chats
- files
- models
- operations
- tunings
- vertexai

--- Debug Analysis ---
❌ 'generate_content' IS NOT in the list. This confirms the error.
   This means the 'client' object from 'genai.GenerativeModel()' in your environment
   is NOT the modern object we expect. It's a different, older 'Client' type.
   The fix is to stop using this object-oriented approach and use a simpler function call.


In [24]:
import pyautogui
print(pyautogui.size())  # Should match your actual screen resolution (e.g., 1920x1080)

Size(width=1920, height=1080)


In [31]:
pyautogui.moveTo(1800, 100)  # Should visibly move the mouse